In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# 定义因果特征提取器
class CausalFeatureExtractor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CausalFeatureExtractor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

# 定义干预器
class Intervention(nn.Module):
    def __init__(self, input_size, output_size):
        super(Intervention, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = self.fc(x)
        return x

# 定义损失函数
def causal_loss(feature_extractor, intervention, x, y):
    z = feature_extractor(x)
    y_pred = intervention(z)
    loss = nn.MSELoss()(y_pred, y)
    return loss

# 加载数据集
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# 训练模型
input_size = 784
hidden_size = 128
output_size = 10
feature_extractor = CausalFeatureExtractor(input_size, hidden_size, output_size)
intervention = Intervention(output_size, output_size)
optimizer = optim.Adam(list(feature_extractor.parameters()) + list(intervention.parameters()), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
        x = x.view(x.shape[0], -1)
        y = torch.nn.functional.one_hot(y, 10).float()  # 将标签转换为one-hot编码
        optimizer.zero_grad()
        loss = causal_loss(feature_extractor, intervention, x, y)
        loss.backward()
        optimizer.step()
        print("Epoch {}, Batch {}, Loss {}".format(epoch, i, loss.item()))

# 预测结果
with torch.no_grad():
    for x, y in test_loader:
        x = x.view(x.shape[0], -1)
        z = feature_extractor(x)
        y_pred = intervention(z)
        acc = (y_pred.argmax(dim=1) == y).float().mean().item()
        print("Accuracy: {}".format(acc))


Epoch 0, Batch 0, Loss 0.162772074341774
Epoch 0, Batch 1, Loss 0.11073499917984009
Epoch 0, Batch 2, Loss 0.10772918164730072
Epoch 0, Batch 3, Loss 0.10205677896738052
Epoch 0, Batch 4, Loss 0.08720044046640396
Epoch 0, Batch 5, Loss 0.0761219710111618
Epoch 0, Batch 6, Loss 0.0776054710149765
Epoch 0, Batch 7, Loss 0.07615017890930176
Epoch 0, Batch 8, Loss 0.07141286879777908
Epoch 0, Batch 9, Loss 0.07486793398857117
Epoch 0, Batch 10, Loss 0.07534658163785934
Epoch 0, Batch 11, Loss 0.06301768124103546
Epoch 0, Batch 12, Loss 0.06129854917526245
Epoch 0, Batch 13, Loss 0.06285395473241806
Epoch 0, Batch 14, Loss 0.06692634522914886
Epoch 0, Batch 15, Loss 0.06285528093576431
Epoch 0, Batch 16, Loss 0.06854448467493057
Epoch 0, Batch 17, Loss 0.05928368493914604
Epoch 0, Batch 18, Loss 0.064654640853405
Epoch 0, Batch 19, Loss 0.05710607022047043
Epoch 0, Batch 20, Loss 0.054957978427410126
Epoch 0, Batch 21, Loss 0.053892217576503754
Epoch 0, Batch 22, Loss 0.05585774779319763
Ep

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import KMeans

# Define the confounder identification-free causal visual feature learning model
class CIFCF(nn.Module):
    def __init__(self, input_size, output_size):
        super(CIFCF, self).__init__()
        self.linear1 = nn.Linear(input_size, 64)
        self.linear2 = nn.Linear(64, output_size)
        
    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x

# Define the global intervention function
def global_intervention(x, model):
    with torch.no_grad():
        y = model(x)
        y_grad = torch.autograd.grad(y, x, retain_graph=True, create_graph=True)[0]
        return y - y_grad * x

# Define the LCICF loss function
def LCICF_loss(y_pred, y_true, g_intervention, model):
    y_intervention = global_intervention(y_true, model)
    return torch.mean((y_pred - y_intervention - g_intervention) ** 2)

# Load the dataset and split it into input and output tensors
X, y = load_dataset()  # Replace with your own dataset loading function
X = torch.Tensor(X)
y = torch.Tensor(y)

# Cluster the dataset into K clusters
K = 5
kmeans = KMeans(n_clusters=K, random_state=0)
cluster_labels = kmeans.fit_predict(X)

# Initialize the model and optimizer
model = CIFCF(X.shape[1], y.shape[1])
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model using the CIFCF algorithm
alpha = 0.1
beta = 0.01
batch_size = 64
num_epochs = 100

for epoch in range(num_epochs):
    cluster_indices = [torch.where(cluster_labels == k)[0] for k in range(K)]
    for k in range(K):
        batch_indices = torch.randint(low=0, high=len(cluster_indices[k]), size=(batch_size,))
        batch_X = X[cluster_indices[k][batch_indices]]
        batch_y = y[cluster_indices[k][batch_indices]]
        
        g_intervention = global_intervention(batch_y, model)
        model.train()
        y_pred = model(batch_X)
        loss = LCICF_loss(y_pred, batch_y, g_intervention, model)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")


NameError: name 'load_dataset' is not defined